In [1]:
import os

import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import cv2
from keras.models import Sequential
from keras.callbacks import EarlyStopping
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.preprocessing import label_binarize
from itertools import cycle
import splitfolders


if str(os.getcwdb()[-3:]).split("'")[1] != 'src':
    for _ in range(2):
        os.chdir(os.path.dirname(os.getcwdb()))


# Proceso

- Se borran las imágenes vacías (".png") y las eliminadas durante el procesado del "dataframe" de características (en el "notebook" llamado "images_data_processing"), y se actualiza paralelamente ese conjunto de datos quitándole las filas correspondientes a las imágenes "png" mencionadas

- El resultado es que en la carpeta de imágenes quedan tantos archivos ".jpg" como filas hay en el "dataframe" de características

In [2]:
df_images_data = pd.read_csv(r'data\processed\images_data_processed.csv', index_col='Id')
raw_data_path = r'data\raw\images'


In [3]:
len(df_images_data)


5701

In [4]:
# Hay varias imágenes vacías. Todas están en formato "png". Hay que eliminarlas antes que nada
no_delete_extension = '.jpg'
for (dirpath, dirnames, filenames) in os.walk(raw_data_path):
    for file in filenames:
        if no_delete_extension not in file:
            if file != 'info_raw.txt':
                os.remove(f'{dirpath}/{file}')


In [5]:
# Se han eliminado muchas filas del "dataframe", con lo que toca borrar las imágenes correspondientes
for (dirpath, dirnames, filenames) in os.walk(raw_data_path):
    for file in filenames:
        if file[:-4] not in df_images_data.index and file[:-4] != 'info_raw':
            os.remove(f'{dirpath}/{file}')


In [6]:
# Se crea una lista de las imágenes definitivas de la carpeta
filenames_list = []
for (dirpath, dirnames, filenames) in os.walk(raw_data_path):
    filenames_list += [file[:-4] for file in filenames if file != "info_raw.txt"]

len(filenames_list)


4519

In [7]:
# Asimismo, hay que borrar del "dataframe" los diamantes que se han sacado de la carpeta por no tener imagen
for index in df_images_data.index:
    if index not in filenames_list:
        try:
            df_images_data.drop(index, inplace=True)
        except Exception:
            print(f'El diamante {index} ya se borró durante la limpieza')

len(df_images_data)


4519

In [8]:
# En el "dataframe" de características de las imágenes solo quedan los mismos diamantes que en la carpeta de imágenes "raw".
sorted(df_images_data.index) == sorted(filenames_list)


True

In [9]:
# En el "dataframe" de características de imagen solo quedan los mismos que en la carpeta de imágenes "raw". Se actualiza el archivo procesado
df_images_data.to_csv(r'data\processed\images_data_processed.csv', index='Id')
